In [1]:
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.models import load_model

2024-02-08 10:59:03.038823: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-08 10:59:03.040832: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-08 10:59:03.080930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-08 10:59:03.082145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-08 10:59:06.533136: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

#### Import neural network, transformers, base input values

In [2]:
# Import the saved neural network
nn_tau_em = load_model("/glade/work/wchuang/mlmicrophysics/tau_run_10/20240206/quantile_neural_net_keras.h5")

# Import the quantile transformers
with open("/glade/work/wchuang/mlmicrophysics/tau_run_10/20240206/output_quantile_transform.pkl", "rb") as out_quant_transform_file:
    output_scaler = pickle.load(out_quant_transform_file)
with open("/glade/work/wchuang/mlmicrophysics/tau_run_10/20240206/input_quantile_transform.pkl", "rb") as in_quant_transform_file:
    input_scaler = pickle.load(in_quant_transform_file)

# Read in the fortran-scam inputs
fortran_directory = "/glade/derecho/scratch/wchuang/scam_ml_port6_emulate8_20240206TEST.arm97/run/"
fortran_input_file = fortran_directory + "test_input.dat"
input = np.fromfile(fortran_input_file, dtype=np.float64, sep=' ').reshape(1, -1)

# Set column names for inputs and outputs
input_col_names = ["qc", "qr", "nc", "nr", "pgam", "lamc", "lamr", "n0r", "rho_clubb"]
output_col_names = ["qctend", "qrtend", "nctend", "nrtend"]

2024-02-08 10:59:14.496988: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


View the various inputs/outputs

In [3]:
pd.DataFrame(input, columns=input_col_names)

,qc,qr,nc,nr,pgam,lamc,lamr,n0r,rho_clubb
0,1.333108e-08,0.0,158.360965,0.0,10.111111,222222.22214,0.0,0.0,0.989029


In [4]:
input_quantile = pd.DataFrame(input_scaler.transform(input), columns=input_col_names)
input_quantile

/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


,qc,qr,nc,nr,pgam,lamc,lamr,n0r,rho_clubb
0,0.0,0.0,0.0,0.0,0.911318,0.344763,0.0,0.0,0.408971


In [5]:
output_quantile = nn_tau_em.predict(input_quantile)
output_quantile

1/1 [==============================] - 0s 97ms/step


array([[0.69710058, 0.2971191 , 0.94884293, 0.94540066]])

In [6]:
output = output_scaler.inverse_transform(output_quantile)
pd.DataFrame(output, columns=output_col_names)

/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


,qctend,qrtend,nctend,nrtend
0,-3.238707e-09,3.247959e-09,-11.251719,35.030717


#### Checking the input quantile scaler

In [7]:
# # Read in the fortran quantile inputs
fortran_input_quantile_file = fortran_directory + "test_quantile_input.dat"
fortran_input_quantile = np.fromfile(fortran_input_quantile_file, dtype=np.float64, sep=' ').reshape(1, -1)
fortran_input_quantile
print(fortran_input_quantile)
# # Quantile transform with the python scaler
input_quantile = input_scaler.transform(pd.DataFrame(input))
print(input_quantile)
# # Compare python-quantile-transformed inputs to the fortran-imported inputs
input_quantile_same = np.allclose(fortran_input_quantile, input_quantile, atol=1e-7)
input_quantile_same
# # Are they the same?
if input_quantile_same:
    print("Arrays are equal within the specified tolerance.")
else:
    print("Arrays are not equal within the specified tolerance.")

[[0.         0.         0.         0.         0.91131816 0.34476323
  0.         0.         0.40897145]]
[[0.         0.         0.         0.         0.91131832 0.344763
  0.         0.         0.40897145]]
Arrays are equal within the specified tolerance.


/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


#### Checking the neural network, which gives us the output quantile values

In [8]:
fortran_output_quantile_file = fortran_directory + "test_quantile_output.dat"
fortran_output_quantile = np.fromfile(fortran_output_quantile_file, dtype=np.float64, sep=' ').reshape(1, -1)
print(fortran_output_quantile)
# Run the quantile transformed inputs through the python nn to get python-quantile-transformed outputs
output_quantile = nn_tau_em.predict(input_quantile)
print(output_quantile)
# Compare the python-quantile-transformed outputs to the fortran-imported-quantile-transformed outputs
output_quantile_same = np.allclose(output_quantile, fortran_output_quantile, atol=1e-7)
# Are these the same?
if output_quantile_same:
    print("Arrays are equal within the specified tolerance.")
else:
    print("Arrays are not equal within the specified tolerance.")

[[0.69710076 0.29711893 0.94884293 0.94540064]]
1/1 [==============================] - 0s 32ms/step
[[0.69710059 0.29711908 0.94884293 0.94540066]]
Arrays are equal within the specified tolerance.


#### Checking the output quantile scaler

In [9]:
# Reverse quantile transform the outputs using python
fortran_output_file = fortran_directory + "test_output.dat"
fortran_output = np.fromfile(fortran_output_file, dtype=np.float64, sep=' ').reshape(1, -1)
print(fortran_output)
# Compare the python outputs to the fortran-imported outputs
output = output_scaler.inverse_transform(output_quantile)
print(output)
# Are they the same?
output_same = np.allclose(output, fortran_output, atol=1e-7)
output_same

[[-3.23870080e-09  3.24795210e-09 -1.12517141e+01  3.50307062e+01]]
[[-3.23870650e-09  3.24795853e-09 -1.12517186e+01  3.50307163e+01]]


/glade/u/home/wchuang/.local/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


True

#### If all checks pass here then the fortran neural network has been implemented successfully
